This code demonstrates the LPME algorithm. LPME assumes the hidden (cost) metric $s$ can be expressed as:
$$ s = <a, r> + c $$.

It uses only oracle comparisons to recover $a$. It works using a binary-search like algorithm on each coordinate.

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import numpy as np
from scipy import spatial

import sys
sys.path.append('../')
from common import Sphere, Oracle, normalize
from lpme import LPME, compute_vector

In [3]:
np.random.seed(7)
nc = 5 # number of classes
r = np.random.uniform(0, 10) # sphere radius
search_tol = 1e-2 # search tolerance

q = nc**2 - nc


In [4]:
# search space is a Sphere
sphere = Sphere(np.random.randn(q), r, q)

# linear performance metric
a = np.random.randn(q)
a = normalize(a)

# B = 0 makes oracle linear
B = np.matrix(np.zeros((q, q)))

In [5]:
oracle = Oracle(a, B)

In [6]:
lpm = LPME(sphere, oracle, search_tol)
ahat = lpm.run_lpme()

In [7]:
a

array([ 0.01250023, -0.19585473,  0.14049531,  0.20815772, -0.25391651,
       -0.02774202, -0.41198427, -0.02981075, -0.39809574, -0.06708321,
        0.08438654,  0.10614552,  0.12328695, -0.31575798, -0.06444732,
       -0.30606313,  0.23728292, -0.22163813, -0.12149291, -0.38505428])

In [8]:
ahat

array([ 0.01227154, -0.19507563,  0.14092336,  0.20683073, -0.25290735,
       -0.02803339, -0.4132031 , -0.02998539, -0.39909754, -0.06738353,
        0.08431556,  0.10503226,  0.12273585, -0.31615512, -0.06481887,
       -0.30597269,  0.23699232, -0.22191356, -0.12176588, -0.38436378])

In [9]:
print("error:", np.linalg.norm(ahat - a))

error: 0.002979974129001106


Code below shows how close $a$ in spherical coordinates (theta_list) is to $ahat$.

In [10]:
def vec_to_theta_list(vec):
    '''
    Converts a vector to theta_list
    '''
    theta_list = []
    cur_sin_product = 1.0
    for i in range(0, len(vec) - 2):
        theta = np.arccos(vec[i] / cur_sin_product) # 0 to pi range
        theta_list.append(theta)
        cur_sin_product *= np.sin(theta)
    
    # 0 to 2pi range
    # possibility 1
    theta_p1 = np.arccos(vec[-2] / cur_sin_product)
    if cur_sin_product * np.sin(theta_p1) - vec[-1] > 1e-2:
        # we need to use the other possibility
        theta_p1 = -theta_p1 + 2 * np.pi
    theta_list.append(theta_p1)
    return theta_list

In [11]:
theta_list_opt = vec_to_theta_list(oracle.a)

In [12]:
# this should be equal to a
# subtract the origin and divide by the radius to get a normalize estimate
(compute_vector(sphere, theta_list_opt) - sphere.origin)/sphere.radius

array([ 0.01250023, -0.19585473,  0.14049531,  0.20815772, -0.25391651,
       -0.02774202, -0.41198427, -0.02981075, -0.39809574, -0.06708321,
        0.08438654,  0.10614552,  0.12328695, -0.31575798, -0.06444732,
       -0.30606313,  0.23728292, -0.22163813, -0.12149291, -0.38505428])

In [13]:
a # equal to above

array([ 0.01250023, -0.19585473,  0.14049531,  0.20815772, -0.25391651,
       -0.02774202, -0.41198427, -0.02981075, -0.39809574, -0.06708321,
        0.08438654,  0.10614552,  0.12328695, -0.31575798, -0.06444732,
       -0.30606313,  0.23728292, -0.22163813, -0.12149291, -0.38505428])

In [14]:
theta_list_opt

[1.5582957755529625,
 1.767940919620489,
 1.4270198156984673,
 1.35461725521667,
 1.8419958741937164,
 1.601179756072418,
 2.0390735108393803,
 1.6074036951098685,
 2.0818274659901883,
 1.6654234173710578,
 1.4511192485406748,
 1.4189550080139803,
 1.3921169209363586,
 2.051677090133744,
 1.6774857276480764,
 2.1043587275585147,
 1.0950925146064312,
 2.0728150476050446,
 4.406753792133734]

In [15]:
# very close to above
# in general: theta_hat - search_tol/2 <= theta_true <= theta_hat + search_tol/2
lpm.theta_list

[1.5585244804918115,
 1.7671458676442586,
 1.4266021327336462,
 1.3560390164909066,
 1.8407769454627694,
 1.6014759425526095,
 2.040194447887903,
 1.6076118657041518,
 2.083145909948701,
 1.6659031356438065,
 1.4511458253398164,
 1.420466209582104,
 1.392854555400162,
 2.052466294190988,
 1.6781749819468916,
 2.1046216409791,
 1.0952622825503475,
 2.073942025221387,
 4.405592822807561]